# CMSC 197 Problem Set 2

Adrian Miguel Custodio

This notebook filters the top 10k dictionary where words occur more than 100 times

In [1]:
##### Standard Libraries #####
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##### For Preprocessing #####
import email  
import re 
from collections import Counter  
from sklearn.metrics import accuracy_score, precision_score, recall_score


## Preprocessing of Data

Split the dataset into three(3) groups: training set for ham, training set for spam, and the testing
set. 

Remove words from the document which may not contribute to the information we want to
extract. These includes dropping the alphanumeric characters and punctuation marks.

Also, remove stop words, more popularly known as meaningless words, from the email body
since those words are not useful in classification as well as reduce the dimensionality of the
dictionary

In [2]:
#setting up the dataset
df_labels = pd.read_csv("labels", sep=" ")
df_labels.columns = ["label", "filepath"]

#change ham labels into 0; spam labels into 1
for i, row in df_labels.iterrows():
    if df_labels['label'][i] == "ham":     
        df_labels.at[i,'filepath'] =  (df_labels.at[i,'filepath']).replace("../", "")
        df_labels.at[i,'label'] = 0
    else:
        df_labels.at[i,'filepath'] =  (df_labels.at[i,'filepath']).replace("../", "")
        df_labels.at[i,'label'] = 1
        
df_labels

,label,filepath
0,0,data/000/000
1,1,data/000/001
2,1,data/000/002
3,0,data/000/003
4,1,data/000/004
...,...,...
37817,1,data/126/017
37818,1,data/126/018
37819,1,data/126/019
37820,1,data/126/020


In [3]:
#open stop_words file
with open("stop_words.txt","r") as file:
    stop_words = file.read().splitlines()


In [4]:
#function the cleans the message and rids it of other special characters
def message_cleanse(message):
    words = ""   
    
    for word in message.split():
        #calls function that stores the things to remove
        msg = word_cleanup(word)
        
        #if message is not found in the stop_words file nor is it empty
        if msg not in stop_words and msg != "" : 
            words += (msg+" ")
            
    return words         



In [5]:
#function that stores the characters to be removed from the messages
def word_cleanup(msg):
    num = "0123456789"
    html_tags = "<[^<>]+>"
    
    msg = msg.lower()
    msg = re.sub(html_tags,'', msg)
    msg = re.sub(num,'', msg)
    msg = re.sub('[^a-z]','', msg)
    
    return msg
    

In [6]:
#function for checking mulitparts of emails
def multipart_checker(message):
        msg = ""
        
        #if message is multipart   
        if message.is_multipart():
            # loops through diff parts of the email
            for email_part in message.walk():
                # checks content whather if text/plain
                if email_part.get_content_type() == "text/plain":
                    # gets the text/plain part
                    msg = email_part.get_payload()  
                    break
        #not multipart
        else:
            msg = message.get_payload()
        
        return msg

In [7]:
#dictionary for the email message and its path
message_dict = {'filepath':[], 'Message':[]}
encode_def = "ISO-8859-1" # default encoding

#for every path in the dataset
for path in df_labels['filepath']:
    message_dict['filepath'].append(path)
    
    # open file for the email content 
    with open(f"{path}", "r", encoding = encode_def) as e_mail:
        
        message = email.message_from_file(e_mail) 
        
        #multipart checker
        msg = multipart_checker(message)
        #message unusdable characters cleaner
        msg_update = message_cleanse(msg)
        
        # append the cleaned message to the content_dict under content 
        message_dict['Message'].append(msg_update)

In [8]:
#convert the dictionary into a dataframe and display it with the df_labels data
df_content = pd.DataFrame.from_dict(message_dict)
df_main= pd.merge(df_labels, df_content, on='filepath')
df_main

,label,filepath,Message
0,0,data/000/000,mailing list queried weeks ago running set arc...
1,1,data/000/001,luxury watches buy rolex rolex cartier bvlgari...
2,1,data/000/002,academic qualifications prestigious nonacc red...
3,0,data/000/003,greetings verify subscription planfans list ch...
4,1,data/000/004,chauncey conferred luscious continued tonsilli...
...,...,...,...
37817,1,data/126/017,great news expec ted infinex ventures infx pri...
37818,1,data/126/018,oil sector going crazy weekly gift kkpt thing ...
37819,1,data/126/019,httpvdtobjdocscaninfo suffering pain depressio...
37820,1,data/126/020,prosperous future increased money earning powe...


In [9]:
#removes the "data/" part of path to use the files numerically
for i, row in df_main.iterrows():
    df_main.at[i,'filepath'] =  (df_main.at[i,'filepath']).replace("data/", "")

#training data set
df_train = df_main[df_main['filepath']<= '071']
df_train

,label,filepath,Message
0,0,000/000,mailing list queried weeks ago running set arc...
1,1,000/001,luxury watches buy rolex rolex cartier bvlgari...
2,1,000/002,academic qualifications prestigious nonacc red...
3,0,000/003,greetings verify subscription planfans list ch...
4,1,000/004,chauncey conferred luscious continued tonsilli...
...,...,...,...
21295,1,070/295,btijclnab binpqnejgmb httpgethighbizez bldb xi...
21296,1,070/296,special offer adobe video collection adobe pre...
21297,1,070/297,doctype html public wcdtd html transitionalen ...
21298,0,070/298,mounted isu infrared demodulator hb realised r...


In [10]:
#training data set of ham
ham_train_set = df_train[df_train['label'] == 0]
ham_train_set.reset_index()

,index,label,filepath,Message
0,0,0,000/000,mailing list queried weeks ago running set arc...
1,3,0,000/003,greetings verify subscription planfans list ch...
2,5,0,000/005,quiet quiet well straw poll plan running
3,6,0,000/006,working departed totally bell labs recommended...
4,10,0,000/010,greetings mass acknowledgement signed planfans...
...,...,...,...,...
7518,21270,0,070/270,equation generate prime numbers equation theor...
7519,21271,0,070/271,equation generate prime numbers equation theor...
7520,21288,0,070/288,dear dmdx users guidance generating dmdx item ...
7521,21293,0,070/293,built handyboard works great testmotor passes ...


In [11]:
#training dataset of spam
spam_train_set = df_train[df_train['label'] == 1]
spam_train_set.reset_index()

,index,label,filepath,Message
0,1,1,000/001,luxury watches buy rolex rolex cartier bvlgari...
1,2,1,000/002,academic qualifications prestigious nonacc red...
2,4,1,000/004,chauncey conferred luscious continued tonsilli...
3,7,1,000/007,nbc today body diet beaches magazines hollywoo...
4,8,1,000/008,oil sector going crazy weekly gift kkpt thing ...
...,...,...,...,...
13772,21294,1,070/294,txtadd
13773,21295,1,070/295,btijclnab binpqnejgmb httpgethighbizez bldb xi...
13774,21296,1,070/296,special offer adobe video collection adobe pre...
13775,21297,1,070/297,doctype html public wcdtd html transitionalen ...


In [12]:
#testing dataset
df_test = df_main[df_main['filepath'] > '071']
df_test.reset_index()

,index,label,filepath,Message
0,21300,1,071/000,hesitantly derive perverse satisfaction clodho...
1,21301,0,071/001,things perform experiment display will remain ...
2,21302,1,071/002,best offer month viggra ci ialis vaiium xa naa...
3,21303,1,071/003,de ar wne cr doesnt matter ow real st mmed ia ...
4,21304,1,071/004,special offer adobe video collection adobe pre...
...,...,...,...,...
16517,37817,1,126/017,great news expec ted infinex ventures infx pri...
16518,37818,1,126/018,oil sector going crazy weekly gift kkpt thing ...
16519,37819,1,126/019,httpvdtobjdocscaninfo suffering pain depressio...
16520,37820,1,126/020,prosperous future increased money earning powe...


In [25]:
#top 
dict_top_10k = {}

for i, r in df_train.iterrows():
    for word in str(r['Message']).split():
        if word in dict_top_10k:
            dict_top_10k[word] += 1
        else:
            dict_top_10k[word] = 1

#sort the dictionary to be from most frequent to least
dict_sort = sorted(dict_top_10k.items(), key=lambda x: x[1], reverse=True)
dict_sort = dict(dict_sort)
top_10k_words= {i : j for i, j in dict_sort.items() if j > 100}

print("Length of Dict where K = 100: ", len(top_10k_words))


Length of Dict where K = 100:  2848


# Creating the Feature Matrices

Create feature matrices for both the spam training set and ham training set with a dimensionality of 2848

In [14]:
#setting the dictionary into a list for index purposes
top_10k_words_list = list(top_10k_words.keys())

#matrix of zeros all 10k words for the ham train set
ham_feat_matrix= np.zeros((len(ham_train_set), 2848))

#for loop to iterate through the ham training set to update the matrix
for i in range(len(ham_train_set)):
    for word in str(ham_train_set.iloc[i]['Message']).split():
        #checks if the word encountered is in the 10k words
        if word in top_10k_words:
            ham_feat_matrix[i][top_10k_words_list.index(word)] = 1
ham_feat_matrix

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
#data frame version of the matrix
ham_df = pd.DataFrame(ham_feat_matrix, columns = top_10k_words)
ham_df

,bb,td,will,width,board,size,price,company,gold,email,...,objects,xc,audio,uf,formal,micro,iaa,cycle,affect,leg
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7520,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#matrix of zeros for all 10k words for the ham train set
spam_feat_matrix = np.zeros((len(spam_train_set), 2848))

#for loop to iterate through the spam training set to update the matrix
for i in range(len(spam_train_set)):
    for word in str(spam_train_set.iloc[i]['Message']).split():
        #checks if the word encountered is in the 10k words
        if word in top_10k_words:
            spam_feat_matrix[i][top_10k_words_list.index(word)] = 1

spam_feat_matrix


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
#data frame version of the matrix
spam_df = pd.DataFrame(spam_feat_matrix, columns = top_10k_words)
spam_df


,bb,td,will,width,board,size,price,company,gold,email,...,objects,xc,audio,uf,formal,micro,iaa,cycle,affect,leg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13774,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13775,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Computing the Priors

Prior probabilities of Ham and Spam

In [18]:
# P (c=ham) = number of ham emails in train set/overall email train set size
prob_ham = ham_train_set.count(axis=0)[0] / df_train.shape[0]  
prob_ham

0.3531924882629108

In [19]:
# P (c=ham) = number of ham emails in train set/overall email train set size
prob_spam = spam_train_set.count(axis=0)[0] / df_train.shape[0]  
prob_spam

0.6468075117370892

# Computing the Likelihood of each word (+ Laplace Smoothing)

In [20]:
#function for laplace smoothing
# equation of probability: (word count + alpha) / (total words + alpha * number of words in vocabulary) 
def laplace_smoothing(feature_matrix,top_10k_words, alpha):
        #sets an array of zeros
        p_count = np.zeros(len(top_10k_words))
        #calculates word count
        word_count = np.sum(feature_matrix, axis= 0)
        #counts the amount of words in the data sets
        total_words = np.sum (word_count)
        
        
        
        #length of the top 10k words dataset
        top_word_len = len(top_10k_words)
        
        #using the equation, calculates the probability of words whether it be ham or spam
        for i in range (top_word_len):
            curr_data = (word_count[i] + alpha) / (total_words + alpha*top_word_len)
            p_count[i] = curr_data
        return p_count
    
alpha = 1
    
ham_word_p = laplace_smoothing(ham_feat_matrix, top_10k_words, alpha)
print("Ham Word Proabilities: ",ham_word_p)

spam_word_p = laplace_smoothing(spam_feat_matrix, top_10k_words, alpha)
print("Spam Word Proabilities: ",spam_word_p)

Ham Word Proabilities:  [6.56096227e-05 5.10297066e-05 8.40167669e-03 ... 2.18698742e-04
 2.11408784e-04 1.82248952e-04]
Spam Word Proabilities:  [3.21091079e-03 2.16058339e-03 6.94856234e-03 ... 3.15413633e-06
 5.36203177e-05 9.46240900e-06]


# Classify Emails

In [21]:
#function for classifying emails
def email_checker (email, prob_ham, prob_spam, ham_word_p, spam_word_p, words_list):
    #initialize variables
    ham_log = 0
    spam_log = 0
    
    #add the initial log probabilities of ham and spam
    ham_log = ham_log + np.log(prob_ham)
    spam_log = spam_log + np.log(prob_spam)
    
    
    #email message split into strings to be tokenized
    sentence = str(email).split()
    
    #for every word in the message
    for word in sentence:
        #if word is in the top 10k words, the probabilities for both ham and spam gets higher
        if word in top_10k_words:
            ham_log =  ham_log + np.log(ham_word_p[words_list.index(word)])
            spam_log = spam_log + np.log(spam_word_p[words_list.index(word)])
    
    #return 0 if email is more likely to be ham, else return 1 for spam
    if ham_log > spam_log:
        return 0
    else:
        return 1

# Testing the Classifier

In [22]:
# using the test dataset to, uh, test the email checker/classifier
df_test['predicted'] = df_test['Message'].apply(lambda msg: email_checker(msg, prob_ham, prob_spam, ham_word_p, spam_word_p , top_10k_words_list))
df_test

C:\Users\airco\AppData\Local\Temp\ipykernel_16804\763495712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predicted'] = df_test['Message'].apply(lambda msg: email_checker(msg, prob_ham, prob_spam, ham_word_p, spam_word_p , top_10k_words_list))


,label,filepath,Message,predicted
21300,1,071/000,hesitantly derive perverse satisfaction clodho...,1
21301,0,071/001,things perform experiment display will remain ...,0
21302,1,071/002,best offer month viggra ci ialis vaiium xa naa...,1
21303,1,071/003,de ar wne cr doesnt matter ow real st mmed ia ...,1
21304,1,071/004,special offer adobe video collection adobe pre...,1
...,...,...,...,...
37817,1,126/017,great news expec ted infinex ventures infx pri...,1
37818,1,126/018,oil sector going crazy weekly gift kkpt thing ...,1
37819,1,126/019,httpvdtobjdocscaninfo suffering pain depressio...,1
37820,1,126/020,prosperous future increased money earning powe...,1


# Performance Evaluation

In [23]:
#turn the categories into array to ensure they are of the same type and display results
actual = np.array(df_test['label']).tolist()
prediction = np.array(df_test['predicted']).tolist()

comparisons_df = pd.DataFrame({"Filepath":df_test['filepath'],"Act Val": actual, "Pred Val": prediction})
comparisons_df


,Filepath,Act Val,Pred Val
21300,071/000,1,1
21301,071/001,0,0
21302,071/002,1,1
21303,071/003,1,1
21304,071/004,1,1
...,...,...,...
37817,126/017,1,1
37818,126/018,1,1
37819,126/019,1,1
37820,126/020,1,1


In [24]:
#utilizing sklearns' accuracy, recall, and precision functions to check their scores
print(f"Accuracy Score= {accuracy_score(actual,prediction)}")
print(f"Recall Score = {recall_score(actual,prediction)}")
print(f"Precision Score = {precision_score(actual,prediction)}")

Accuracy Score= 0.9306379372957269
Recall Score = 0.9259991019308487
Precision Score = 0.9697169190256748
